The Algorithm -

1. Stating the iteration parameters and the model parameters
2. Stating steady state values of parameters
3. Calculating steady state of variables using the equations and constants derived
4. Using the data and finding h_ast, k_ast and y_ast
5. Stating the trajectory of exogenous variables
6. Using shooting algorithm to find saddle point and checking for convergence of path to steady state.
7. After convergence, plotting the required graphs using our model and the data values to compare both. 


In [ ]:
import numpy as np
import scipy.optimize as opt
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
import scipy.linalg as la
import numpy as np
from numpy.linalg import matrix_rank

import sympy as sym
from sympy import *
from sympy import symbols, Matrix, Transpose
from sympy import lambdify

import math
import cmath
from math import *

import mpmath as mp
from math import sqrt
from matplotlib import rcParams



In [ ]:
# Parameter values

# Iteration parameters
S = 100 # number of periods in the shooting
niter=200 # number of possible trials of the intial value
epsilon=.00005 #convergence criterion

# Model Parameters
beta = 0.933 # Discount factor
theta = 0.3 # Share of capital in output rents
alpha = 3.2 # disutility from labor parameter
delta = 0.035 # Capital depreciation rate
T =5*24 # Total number of hours owned in a week


In [ ]:

gamma = 1.014 # TFP to the power 1/(1-theta) growth rate. Long run growth rate of per capita income 
gA_ast = gamma**(1-theta) # = (1+g) in our model. TFP growth rate
n_ast = 1.014 # population growth rate. n_ast = (1+n) in our model

chi_ast = 0.176 # long-run government share
tau_ast = 0.098 # long-run capital tax


N0 = 10900104
H0 = 23.0345983*N0
k0 = 2.47**(1/(1-theta))*(H0/N0)
A0 = 1**(1-theta)


In [ ]:
# Steady State
c1 = (((gA_ast**(1/(1-theta))/beta - 1)*(1/(1-tau_ast))+ delta)*1/theta)**(1/(theta - 1)) # steady state k/h ratio
c2 = (1 - chi_ast)*c1**(theta - 1) - (gA_ast**(1/(1-theta))*n_ast - 1 + delta)  # steady state c/k ratio
c3 = c2*alpha/((1-theta)*c1**(theta-1)) # steady state (T - h)/h
h_ast = 1/(1+c3)*T
k_ast = c1*h_ast
y_ast = k_ast**theta*h_ast**(1-theta)/c1**(theta-1)
c_ast = c2*y_ast

print("[k0, k_ast, c_ast, h_ast] = ", k0, k_ast, c_ast, h_ast)

In [ ]:
# Trajectory of Exogenous Variables


chi_t = [0.093, 0.112, 0.098, 0.097, 0.088, 0.091, 0.085, 0.078, 0.087, 0.094, 0.101, 0.106, 0.113, 0.114, 0.115, 0.112, 0.102, 0.091, 0.09, 0.097, 0.099, 0.097, 0.1, 0.104, 0.15, 0.155, 0.189, 0.209, 0.212, 0.231, 0.172, 0.173, 0.168, 0.165, 0.166, 0.166, 0.163, 0.162, 0.161, 0.173, 0.176]
Chi = [1]*S 
for i in range(0, S):
    if i < len(chi_t):
        Chi[i] = chi_t[i]
    else:
        Chi[i] = chi_t[len(chi_t)-1]

tau_t = [0.037, 0.043, 0.034, 0.035, 0.036, 0.039, 0.038, 0.028, 0.045, 0.027, 0.029, 0.03, 0.029, 0.026, 0.026, 0.026, 0.027, 0.031, 0.039, 0.039, 0.04, 0.063, 0.063, 0.057, 0.058, 0.057, 0.061, 0.06, 0.057, 0.074, 0.074, 0.082, 0.089, 0.09, 0.096, 0.099, 0.105, 0.105, 0.108, 0.111, 0.098]
Tau = [1]*S 
for i in range(0, S):
    if i < len(tau_t):
        Tau[i] = tau_t[i]
    else:
        Tau[i] = tau_t[len(tau_t)-1]

        
n_t = [1.031, 1.032, 1.033, 1.033, 1.033, 1.035, 1.033, 1.032, 1.033, 1.034, 1.031, 1.033, 1.034, 1.032, 1.029, 1.027, 1.025, 1.024, 1.024, 1.024, 1.024, 1.024, 1.024, 1.024, 1.024, 1.023, 1.024, 1.024, 1.023, 1.022, 1.022, 1.021, 1.02, 1.02, 1.02, 1.018, 1.018, 1.018, 1.017, 1.016, 1.015]
n = [1]*S 
for i in range(0, S):
    if i < len(n_t):
        n[i] = n_t[i]
    else:
        n[i] = n_t[len(n_t)-1]
        
A_t = [1, 1.04378473, 1.101607201, 1.150984455, 1.181143954, 1.137405416, 1.146534393, 1.140791121, 1.214149745, 1.236197781, 1.268104407, 1.234881423, 1.218360599, 1.243219825, 1.283024964, 1.30214835, 1.361505992, 1.364407932, 1.369758769, 1.338515507, 1.370358575, 1.299345982, 1.351457244, 1.34782411, 1.360881802, 1.37488693, 1.374290051, 1.41595458, 1.394709054, 1.301317914, 1.249561069, 1.260135895, 1.249347697, 1.25594754, 1.339332806, 1.375978959, 1.483980171, 1.55773101, 1.554106291, 1.488266754, 1.496353864]
gA = [1]*S #gA is (1+g) in our model. TFP growth rate
for i in range(0, S):
    if i < len(A_t):
        gA[i] = (A_t[i]/A_t[i-1])**(1-theta)
    else:
        gA[i] = (A_t[len(A_t)-1]/A_t[len(A_t)-2])**(1-theta)
        
    

gamma = [1]*S # TFP to the power 1/(1-theta) growth rate. Long run growth rate of per capita income 
for i in range(0, S):
    if i < len(A_t):
        gamma[i] = (A_t[i]/A_t[i-1])
    else:
        gamma[i] = (A_t[len(A_t)-1]/A_t[len(A_t)-2])
        
A = [1]*S
for i in range(0, S):
    if i < len(A_t):
        A[i] = A_t[i]**(1-theta)
    else:
        A[i] = A_t[len(A_t)-1]**(1-theta)
    

hh_t = [23.829, 23.772, 23.685, 23.581, 23.474, 23.886, 23.734, 23.946, 23.76, 23.581, 23.187, 23.401, 22.929, 22.103, 21.525, 21.378, 21.472, 21.929, 22.116, 22.535, 22.508, 23.418, 23.144, 24.134, 24.642, 25.148, 24.718, 24.601, 23.997, 22.937, 23.189, 22.888, 22.896, 23.177, 23.548, 24.011, 24.345, 24.812, 24.783, 24.388, 24.645]
hh = [1]*S 
for i in range(0, S):
    if i < len(hh_t):
        hh[i] = hh_t[i]
    else:
        hh[i] = hh_t[len(hh_t)-1]
        
    
K_to_Y_data = [2.477, 2.402, 2.357, 2.32, 2.331, 2.391, 2.45, 2.477, 2.452, 2.508, 2.497, 2.544, 2.611, 2.607, 2.599, 2.573, 2.477, 2.502, 2.526, 2.603, 2.572, 2.652, 2.6, 2.557, 2.534, 2.513, 2.603, 2.514, 2.647, 2.914, 2.954, 2.923, 2.835, 2.813, 2.762, 2.703, 2.621, 2.576, 2.597, 2.701, 2.696]
K_to_Y_data= [1]*S 
for i in range(0, S):
    if i < len(K_to_Y_data):
        K_to_Y_data[i] = K_to_Y_data[i]
    else:
        K_to_Y_data[i] = K_to_Y_data[len(K_to_Y_data)-1]
        
        
saving_rate = [0.163, 0.132, 0.152, 0.172, 0.192, 0.167, 0.19, 0.211, 0.2, 0.191, 0.192, 0.16, 0.136, 0.141, 0.149, 0.162, 0.218, 0.193, 0.199, 0.185, 0.201, 0.198, 0.151, 0.162, 0.182, 0.18, 0.147, 0.131, 0.123, 0.119, 0.122, 0.107, 0.115, 0.123, 0.141, 0.152, 0.166, 0.167, 0.178, 0.173, 0.171]
saving_rate = [1]*S 
for i in range(0, S):
    if i < len(saving_rate):
        saving_rate[i] = saving_rate[i]
    else:
        saving_rate[i] = saving_rate[len(saving_rate)-1]


In [ ]:
Time = np.linspace(1970, 2069, 100)
TT = len(Time)
fig = plt.figure()
ax1 = fig.add_subplot(1,1,1)
ax1.plot(Time, A[0:TT], 'k')
ax1.set_title('Productivity Growth')
ax1.set_xlabel('Period')
plt.axis([1970,2030, .8, max(max(A[0:TT]), max(A[0:TT])) + 0.25])

fig = plt.figure()
ax1 = fig.add_subplot(1,1,1)
ax1.plot(Time, n[0:TT], 'k')
ax1.set_title('Population Growth')
ax1.set_xlabel('Period')
plt.axis([1970, 2030, 1,1.04])
#plt.axis([1970,2030, .8, max(max(A[0:TT]), max(A[0:TT])) + 0.25])


In [ ]:
# Initial Conditions for capital

k = [1]*(S)
c = [1]*(S)
h = [1]*(S)

k[0] = k0

#Initial condition for consumption
c_0_L = .05 *c_ast
c_0_H = 1.05*c_ast


#Solving for k_t+1 and c_t+1 and h_t+1 until convergence along the saddle path
niter = 500
S = 100
flag  = 0

#Start iteration
for j in range(0,niter):
    c[0] = (c_0_L+c_0_H)/2 #Initial guess for c_o
    
    def func(x): 
        return T - (k[0]/x)**(-theta)*(alpha*c[0]/(1-theta)) - x #Consumption-Leisure choice

    root = fsolve(func, 15)
    h[0] = root[0]
    
    
    #Generate paths for c and k given intitial guess for c_0
    for t in range(0,S-1):
        k[t+1] = (k[t]**theta*h[t]**(1-theta)*(1- Chi[t]) +(1-delta)*k[t]-c[t])/((gamma[t])*n[t])
        #print(k[t+1])
        
        
        if k[t+1] < 0:
            flag = 1
            #print(flag)
            break
        if k[t+1]>2*k_ast:
            flag =2
            #print(flag)
            break
            
        def cfunc(z):
            # Solves for c_t+1 and h_t+1 simultaneously
            fc = z[0]
            fh = z[1]
            f1 = T - (alpha*fc/(1-theta))*(k[t+1]/fh)**(-theta) - fh 
            f2 = fc/c[t] - (beta/gamma[t])*(1 + (1-Tau[t+1])*(theta*((k[t+1]/fh)**(theta-1)) - delta)) 
            return [f1, f2]
        
        sol = fsolve(cfunc, [15, 15])
        
        c[t+1] = sol[0]
        h[t+1] = sol[1]
        
        # Check for convergence
        # Update the intitial value if not convergence
        if t== S-2:
            
            if (abs(k[S-1]-k_ast)/k_ast < epsilon) and abs((c[S-1] -c_ast)/c_ast < epsilon):
                flag = 10
                print( "Converged in iteration ", j)
                break
            # Update initial guess for c
            elif (k[S-1]-k_ast)/k_ast <=epsilon:
                flag =1
            elif (k[S-1] - k_ast)/k_ast >= epsilon:
                flag =2
        
        
    if flag == 1:
        c_0_H = c[0]
        
    elif flag == 2:
        c_0_L = c[0]
        
    elif flag == 10:
        break
        


        #End iteration loop
        #End shooting algorithm if reached max number of iterations
    
if j == niter-1:
    print('Failure in Convergence!')


In [ ]:
t = S-2
print(k[t+1], c[t+1], h[t+1])
print([k_ast, c_ast, h_ast])
print (c[0])
print (h[0])
print(epsilon)
print((k[S-1]-k_ast)/k_ast <epsilon)
print((c[S-1]-c_ast)/c_ast <epsilon)
print((h[S-1]-h_ast)/h_ast)
print ((c[S-1]-c_ast)/c_ast)

print (c[S-1])

In [ ]:
KY = [2.5]*S
for t in range(S):
    KY[t] = (k[t]**(theta-1)*(h[t])**(1-theta))
    

In [ ]:
#Plotting normalised c, h, k and savings rate 
k_s0 = [k_ast]*(S)
h_s0 = [h_ast]*(S)
c_s0 = [h_ast]*(S) 


fig = plt.figure()
ax1 = fig.add_subplot(1,1,1)
ax1.plot(Time, KY, 'k-')
ax1.set_title('K/Y')
ax1.set_xlabel('Period')
ax1.set_xlim(1970,2030)




In [ ]:

k_i = [1]*(S)
c_i = [1]*(S)
h_i = [1]*(S)

k_i[0] = k[0]
c_i[0] = c[0]


for t in range (0, S):
    co = c[t]
    ko = k[t]
    Tauo = Tau[t]
    
    def func(h): 
        return h - T + (k[0]/h)**(-theta)*(alpha*c[0]/(1-theta))

    kit = fsolve(func, 20)
    h_i[t] = kit[0]
    
    if t == len(Chi) -1:
        break
        
    else:
        k_i[t+1] = (ko**theta*h_i[t]**(1-theta)*(1- Chi[t]) +(1-delta)*ko-co)/((gamma[t])*n[t])
        kn = k_i[t+1]
        At = A[t]
        Taut = Tau[t+1]
        Tau_t = Tau[t]
    
            
        def cff(z):
            # Solves for c_t+1 and h_t+1 simultaneously
            gc = z[0]
            gh = z[1]
            g1 = T - alpha*gc/((1-theta)*(kn/gh)**(theta)) - gh 
            g2 = gc/co - (beta/gamma[t])*(1 + (1-Taut)*(theta*((kn/gh)**(theta-1)) - delta)) 
            return [g1, g2]
        
        sol = fsolve(cff, [15, 15])
        
        c_i[t+1] = sol[0]
        h_i[t+1] = sol[1]
    

        